
# **Analisa Kepuasan Pelanggan Marketplace berdasarkan Review**


# <span id="1"></span> Overview
<hr/>
Mencoba semua metode untuk memprediksi kepuasan pelanggan berdasarkan review setiap produk dan sentiment menggunakan Naive Bayes
<br/>


## Import modules dan Load dataset dari kaggle


In [3]:
import json
import os
import kaggle
from dotenv import load_dotenv

load_dotenv()

token_api = {"username":"maxxoto","key":""}

token_api['key'] = os.getenv('KAGGLE_API_KEY')

# Buat JSON File
with open('kaggle.json', 'w') as outfile:
    json.dump(token_api, outfile)

%mv kaggle.json /Users/maxxoto/.kaggle/kaggle.json

# Change the permissions of the kaggle.json file
os.chmod('/Users/maxxoto/.kaggle/kaggle.json', 0o600)

!kaggle datasets download -d maxxoto/customer-review-elektronik-lazada-dataset

Dataset URL: https://www.kaggle.com/datasets/maxxoto/customer-review-elektronik-lazada-dataset
License(s): unknown
 91%|██████████████████████████████████▋   | 6.00M/6.57M [00:01<00:00, 5.83MB/s]
100%|██████████████████████████████████████| 6.57M/6.57M [00:01<00:00, 4.99MB/s]


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Module metode

# from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import sklearn.metrics as mt

# from plotly import tools
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
import plotly
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



# Load Data
df1 = pd.read_csv('customer-review-elektronik-lazada-dataset.zip', compression='zip', header=0, sep=',', quotechar='"')
# df1 = pd.read_csv('Dataset AI - Reviews.csv')


# DATA CLEANING
df = df1[['itemName','Brand','rating','reviewContent','clientType']]

df1.info()
# df1.describe()
df1.head(20)

## Menambahkan perhitungan tiap kata pada review

In [ ]:
# fill NA values by space
df['reviewContent'] = df['reviewContent'].fillna('')

# CountVectorizer() converts a collection
# of text documents to a matrix of token counts
vectorizer = CountVectorizer()
# assign a shorter name for the analyze
# which tokenizes the string
analyzer = vectorizer.build_analyzer()

def wordcounts(s):
    c = {}
    # tokenize the string and continue, if it is not empty
    if analyzer(s):
        d = {}
        # find counts of the vocabularies and transform to array
        w = vectorizer.fit_transform([s]).toarray()
        # vocabulary and index (index of w)
        vc = vectorizer.vocabulary_
        # items() transforms the dictionary's (word, index) tuple pairs
        for k,v in vc.items():
            d[v]=k # d -> index:word
        for index,i in enumerate(w[0]):
            c[d[index]] = i # c -> word:count
    return  c

# add new column to the dataframe
df['Word Counts'] = df['reviewContent'].apply(wordcounts)
df.head()

## Mencetak banyaknya kata yang muncul setiap reviewnya


In [ ]:
selectedwords = ['good','good job','bagus','puas','memuaskan','recommended','joss','mantap',
                 'keren','sesuai','barang sesuai','mengecewakan','rusak','tidak bagus','parah','pengiriman lama',
                 'bekas','kurang puas','lambat','keterlaluan','kecewa','barang tidak sesuai','tidak diterima',
                 'tidak menerima barang','packing pecah','barang hancur','penipu','bohong','fast respon']



def selectedcount(dic,word):
    if word in dic:
        return dic[word]
    else:
        return 0

# Copy dataframe utama ke dataframe wordcount untuk diolah tanpa merusak dataframe utama
dfwc = df.copy()

for word in selectedwords:
    dfwc[word] = dfwc['Word Counts'].apply(selectedcount,args=(word,))

word_sum = dfwc[selectedwords].sum()
print('Selected Words')
print(word_sum.sort_values(ascending=False).iloc[:5])

# Masukkan hasil hitungan class name / kategory ke wordcloud
print('\nBrand')
print(df['Brand'].fillna("Empty").value_counts().iloc[:5])

fig, ax = plt.subplots(1,2,figsize=(20,10))
wc0 = WordCloud(background_color='white',
                      width=450,
                      height=400 ).generate_from_frequencies(word_sum)

cn = df['Brand'].fillna(" ").value_counts()
wc1 = WordCloud(background_color='white',
                      width=450,
                      height=400
                     ).generate_from_frequencies(cn)

ax[0].imshow(wc0)
ax[0].set_title('Kata Pilihan\n',size=25)
ax[0].axis('off')

ax[1].imshow(wc1)
ax[1].set_title('Brand\n',size=25)
ax[1].axis('off')

rt = df['reviewContent']
plt.subplots(figsize=(18,6))
wordcloud = WordCloud(background_color='white',
                      width=900,
                      height=300
                     ).generate(" ".join(rt))
plt.imshow(wordcloud)
plt.title('Rekap Kata Reviewer\n',size=25)
plt.axis('off')
plt.show()

 ## Melihat keterkaitan suatu data dari kolom : Rating dan Brand


In [ ]:
df1=df['rating'].value_counts().to_frame()
avgdf1 = df.groupby('Brand').agg({'rating': np.average})
avgdf1.describe()
trace1 = go.Bar(
    x=avgdf1.index,
    y=round(avgdf1['rating'],2),
    marker=dict(
        color=df['rating'],
        colorscale = 'RdBu')
)

trace2 = go.Bar(
    x=df1.index,
    y=df1.rating,
    marker=dict(
        color=df1['rating'],
        colorscale = 'RdBu')
)


fig = tools.make_subplots(rows=2, cols=1, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)


fig['layout']['xaxis1'].update(title='Brand')
fig['layout']['yaxis1'].update(title='Rata-rata Rating')
fig['layout']['xaxis2'].update(title='Rating')
fig['layout']['yaxis2'].update(title='Jumlah')


fig['layout'].update(height=1000, width=1200,showlegend=False)
fig.update_layout({'plot_bgcolor':'rgba(0,0,0,0)',
                   'paper_bgcolor':'rgba(0,0,0,0)'})
fig.show()

#fig['layout'].update(plot_bgcolor='rgba(0,0,0,0)')
#fig['layout'].update(paper_bgcolor='rgba(0,0,0,0)')
# py.iplot(fig)
# plotly.offline.plot(fig)

# Visualisasi Penjualan terbanyak

In [ ]:
cv = df['Brand'].value_counts()

trace = go.Scatter3d( x = avgdf1.index,
                      y = avgdf1['rating'],
                      z = cv[avgdf1.index],
                      mode = 'markers',
                      marker = dict(size=10,color=avgdf1['rating']),
                      hoverinfo ="text",
                      text="Brand: "+avgdf1.index+" \ Rating rata-rata: "+avgdf1['rating'].map(' {:,.2f}'.format).apply(str)+" \ Jumlah Reviewer: "+cv[avgdf1.index].apply(str)
                      )

data = [trace]
layout = go.Layout(title="Rata-rata rating,brand dan jumlah reviewer",
                   scene = dict(
                    xaxis = dict(title='Brand'),
                    yaxis = dict(title='Rata-rata Rating'),
                    zaxis = dict(title='Jumlah Penjualan'),),
                   margin = dict(l=30, r=30, b=30, t=30))
fig = go.Figure(data=data, layout=layout)
fig.show()
plotly.offline.plot(fig)


 # Membuat Klasifikasi


Membuat sentiment dikolom baru untuk menentukan apakah review tersebut positif atau negatif berdasarkan point .

1-2 = Negatif
3 = Netral
4-5 = Positif

In [ ]:
#Memfilter menghilangkan rating bernilai 3
df = df[df['rating'] != 3]

#membuat kolom baru dengan kriteria jika rating > 4 sentiment = True
df['Sentiment Aktual'] = df['rating'] >=4
df.head(10)

# KOLOM SENTIMENT YAITU DEPENDENT VARIABEL YANG DIGUNAKAN UNTUK DATA TESTING DAN TRAINING
# UNTUK PEMBANDING SAAT PROSES TRAINING DATA
# https://www.dailysmarty.com/posts/difference-between-independent-and-dependent-variables-in-machine-learning

# Membagi data training dan testing , dengan library
train_data,test_data = train_test_split(df,train_size=0.8,random_state=0)

# prepare data training
X_train = vectorizer.fit_transform(train_data['reviewContent'])
y_train = train_data['Sentiment Aktual']

# prepare data testing
X_test = vectorizer.transform(test_data['reviewContent'])
y_test = test_data['Sentiment Aktual']

## <span id="8"></span> Naive Bayes

In [ ]:
start=dt.datetime.now()
nb = MultinomialNB()
nb.fit(X_train,y_train)
print('Elapsed time: ',str(dt.datetime.now()-start))

# <span id="11"></span> Proses Hasil Prediksi


## <span id="12"></span> Tambah Hasil Prediksi ke Data Frame

In [ ]:
# define a dataframe for the prediction probablities of the models
# df1 = train_data.copy()
# df1['Logistic Regression'] = lr.predict_proba(X_train)[:,1]
# df1['Naive Bayes'] = nb.predict_proba(X_train)[:,1]
# df1['SVM'] = svm.decision_function(X_train)
# df1['Neural Network'] = nn.predict_proba(X_train)[:,1]
# df1=df1.round(2)
# df1.head(20)

#define a dataframe for the predictions
df2 = train_data.copy()
# df2['Logistic Regression'] = lr.predict(X_train)
df2['Sentiment Prediksi'] = nb.predict(X_train)
# df2['SVM'] = svm.predict(X_train)
# df2['Neural Network'] = nn.predict(X_train)
df2.sample(20)

## <span id="11"></span> Kesimpulan Analisa Berdasarkan Prediksi Sentiment

In [ ]:
## <span id="11"></span>hKesimpulan </span>
df_hasil = df2['Sentiment Prediksi']
df_aktual = df2['Sentiment Aktual']
# df['Class Name'].fillna("Empty").value_counts().iloc[:5]
total_data = df_hasil.count()
# TOTAL PREDIKSI
total_positif = df_hasil.values.sum()
total_negatif = (~df_hasil).values.sum()

#TOTAL AKTUAL
total_aktual_positif = df_aktual.values.sum()
total_aktual_negatif = (~df_aktual).values.sum()

print("Jumlah Total Data :" , total_data)
print("Jumlah Positif : " , total_positif)
print("Jumlah Negatif : " , total_negatif)

# Visualisasi Presentase

color_palette_list = ['#009ACD', '#ADD8E6', '#63D1F4', '#0EBFE9',
                      '#C1F0F6', '#FFA500','#FFFF00']
labels = ['Pelanggan Puas',
          'Pelanggan Tidak Puas']
percentages = [total_positif, total_negatif]
percentages2 = [total_aktual_positif,total_aktual_negatif]

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(20,15))
ax1.pie(percentages,labels = labels,colors = color_palette_list[0:2],autopct = '%1.1f%%') #plot first pie
ax1.set_title('Presentase Data Prediksi')

ax2.pie(percentages2,labels = labels,colors = color_palette_list[5:7],autopct = '%1.1f%%') #plot second pie
ax2.set_title('Presentase Data Aktual')

# data = [trace]


# fig = go.Figure(data=data, layout=layout)
plt.show()
# plotly.offline.iplot(fig, filename='lazada_kesimpulan')

## Menghitung kinerja klasifikasi dengan ROC Curves and AUC

Menampilkan performa dari hasil binary klasifikasi

In [ ]:
# pred_lr = lr.predict_proba(X_test)[:,1]
# fpr_lr,tpr_lr,_ = roc_curve(y_test,pred_lr)
# roc_auc_lr = auc(fpr_lr,tpr_lr)

pred_nb = nb.predict_proba(X_test)[:,1]
fpr_nb,tpr_nb,_ = roc_curve(y_test.values,pred_nb)
roc_auc_nb = auc(fpr_nb,tpr_nb)

# pred_svm = svm.decision_function(X_test)
# fpr_svm,tpr_svm,_ = roc_curve(y_test.values,pred_svm)
# roc_auc_svm = auc(fpr_svm,tpr_svm)

# pred_nn = nn.predict_proba(X_test)[:,1]
# fpr_nn,tpr_nn,_ = roc_curve(y_test.values,pred_nn)
# roc_auc_nn = auc(fpr_nn,tpr_nn)

# f, axes = plt.subplots(2, 2,figsize=(15,10))
f, axes = plt.subplots(figsize=(15,10))
# axes[0,0].plot(fpr_lr, tpr_lr, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_lr))
# axes[0,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# axes[0,0].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
# axes[0,0].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Logistic Regression')
# axes[0,0].legend(loc='lower right', fontsize=13)

axes.plot(fpr_nb, tpr_nb, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_nb))
axes.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Naive Bayes')
axes.legend(loc='lower right', fontsize=13)


# axes[1,0].plot(fpr_svm, tpr_svm, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_svm))
# axes[1,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# axes[1,0].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
# axes[1,0].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Support Vector Machine')
# axes[1,0].legend(loc='lower right', fontsize=13)

# axes[1,1].plot(fpr_nn, tpr_nn, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_nn))
# axes[1,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# axes[1,1].set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
# axes[1,1].set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Neural Network')
# axes[1,1].legend(loc='lower right', fontsize=13);

## Evaluasi Model dengan Confusion Matrix

In [ ]:
# preparation for the confusion matrix
# lr_cm=confusion_matrix(y_test.values, lr.predict(X_test))
nb_cm=confusion_matrix(y_test.values, nb.predict(X_test))
# svm_cm=confusion_matrix(y_test.values, svm.predict(X_test))
# nn_cm=confusion_matrix(y_test.values, nn.predict(X_test))


plt.figure(figsize=(15,12))
plt.suptitle("Confusion Matrix",fontsize=24)

# plt.subplot(2,2,1)
# plt.title("Logistic Regression")
# sns.heatmap(lr_cm, annot = True,fmt=".0f");

plt.subplot(2,2,2)
plt.title("Naive Bayes")
sns.heatmap(nb_cm, annot = True,fmt=".0f");

# plt.subplot(2,2,3)
# plt.title("Support Vector Machine (SVM)")
# sns.heatmap(svm_cm, annot = True, fmt=".0f");

# plt.subplot(2,2,4)
# plt.title("Neural Network")
# sns.heatmap(nn_cm, annot = True, fmt=".0f");

## Precision - Recall - F1-Score

Precision Recall

$$\textbf{Precision} = \dfrac{TP}{TP + FP} \;\;\;\;\;\;\;\;\;\; \textbf{Recall} = \dfrac{TP}{TP + FN}$$


$$\textbf{}$$


$$\textbf{F}_{1} = 2 \; \dfrac{Precision \; \times \;Recall}{Precision + Recall} = \dfrac{2 \; TP}{2 \; TP + FN + FP}$$



In [ ]:
print("\n Naive Bayes")
print(mt.classification_report(y_test, nb.predict(X_test)))
